# Evaluating Impact of Treatment (B Version) on Conversion Rate

This program expects to find the data at 'holdout_data.csv' in the same folder.

In order to answer the prompts, I made the following assumptions:
* Complete duplicates should be deduplicated. 36,893 duplicate records were removed.
* If a unique **userhash** appears in records under both treatment and control, it counts as a unique user for both. It only counts towards a **group**'s conversion rate if the conversion record comes in under that **group**. Overall, 26,426 **userhash** values count towards both treatment and control totals. At the cohort level, only 92 **userhash** values appear in both treatment and control within the same cohort.
* If a **sessionid** is tied to multiple (sometimes unique) **bidprice_usd** values, use the largest one to calculate VPT.

Please see the other program, for investigation of duplications.

# Data Dictionary

| Field | Type | Definition |
| ----- | ---- | ---------- |
| agegroup | str | age buckets ranging from 18-25, 26-30, 31-35 to 65+ |
| bidprice_usd | float | value paid by advertiser to publisher |
| campaigntimestamp | datetime | time of event (user interaction) |
| cohort | int | experiment was divided into distinct cohorts which each correspond to one month; format 'YYYYMM'|
| device | str | device ∈ ('desktop', 'mobile', 'tablet', 'other', nan) |
| gender | str | gender ∈ ('f', 'm', nan) | 
| group | str | group ∈ ('treatment', 'control') users in treatment group saw an ad at checkout, users in control did not |
| sessionid | str | uniquely identifies sessions; a user can convert at most once per session |
| userhash | str | uniquely identifies users |
| value | float | revenue value in USD; NaN when user did not convert, >0 when user converted |
| verticalname | str | verticalname ∈ ('Retail', 'Media and Entertainment', 'Food and Beverage', 'Ticketing', 'Travel', 'Finance', 'Sports and Fitness'); vast majority of records fall under first four verticals |

In [8]:
import numpy as np
from scipy import stats
from math import sqrt

ModuleNotFoundError: No module named 'scipy'

# Import Data from BigQuery

* Deleted record with invalid cohort, campaigntimestamp value directly in BigQuery.

* In dbt:
    * Renamed columns
    * Standardized device values by making all lower-case
    * Deduped records

Thus, old load_and_clean function is not needed.

In [2]:
from google.cloud import bigquery
import pandas as pd
import db_dtypes

In [3]:
client = bigquery.Client()

query = """
    SELECT *
    FROM holdout_data.sessions
"""

In [4]:
df = client.query(query).result().to_dataframe()

How many unique users (identified by userhash) do we have per vertical, per cohort?

In [4]:
# unique users per vertical/cohort
uniq_pivot = pd.pivot_table(
    data, values = 'userhash', index = 'verticalname', columns = 'cohort', aggfunc= 'nunique', margins=True
).fillna(0).astype('int')
uniq_pivot

cohort,202002,202003,202004,202005,202006,All
verticalname,,,,,,
Finance,95,3,57,37,118,306
Food and Beverage,2593,6172,41317,70382,40135,156391
Loyalty and Samples,19,21,31,31,32,129
Media and Entertainment,40047,50347,83418,92593,140618,328704
Retail,9606,12996,17010,21235,64911,118424
Sports and Fitness,1,0,0,0,0,1
Ticketing,39412,8069,2495,4691,11127,64208
Travel,5725,3335,141,208,2565,11637
All,97498,80943,144469,189176,259506,671795


In [5]:
from pivottablejs import pivot_ui

ModuleNotFoundError: No module named 'pivottablejs'

# Overall Metrics

## Conversion Rate

In [31]:
def calculate_conv_rate(df, group, fil=None):
    
    # filter to specified group
    mask = df['group'] == group
    data = df[mask]
    
    # filter for additional values if specified
    if fil:
        for k, v in fil.items():
            data = data[data[k] == v]
    
    # group by userhash to get list of unique users with their total conversion value
    unique_users = data.groupby('userhash')['value'].sum()
  
    # total number of unique users (sample size)
    n = len(unique_users)
    
    if n == 0:
        return 1, 0, 0
    
    # number of unique users that converted (positive 'value' sum)
    n_conv = len(unique_users[unique_users > 0])
    
    conv_rate = n_conv / n
    
    return conv_rate, n_conv, n

In [32]:
# total number of unique users
n_overall = data['userhash'].nunique()

# total number of unique users who converted
n_conv_overall = data[data['value'] > 0]['userhash'].nunique()

# calculate conversion rate, number of unique converters, and number of unique users for treatment group
conv_rate_T_overall, n_conv_T_overall, n_T_overall = calculate_conv_rate(data, 'treatment')

# calculate the same for control group
conv_rate_C_overall, n_conv_C_overall, n_C_overall = calculate_conv_rate(data, 'control')

# calculate overall uplift
uplift_overall = (conv_rate_T_overall - conv_rate_C_overall) / conv_rate_C_overall

### Results

In [33]:
print(
    f" {conv_rate_C_overall * 100:.2f}% of unique users in the control group converted.\
    \n {conv_rate_T_overall * 100:.2f}% of unique users in the treatment group converted.\
    \n The campaign yielded {uplift_overall * 100:.2f}% uplift in conversion rate."
)

 1.95% of unique users in the control group converted.    
 2.23% of unique users in the treatment group converted.    
 The campaign yielded 13.93% uplift in conversion rate.


## Value Per Transaction

In [24]:
def calculate_vpt(df, fil=None, dedupe=True):
    
    # filter additional values if specified
    if fil:
        for k, v in fil.items():
            df = df[df[k] == v]
    
    if dedupe:
        # assuming each sessionid should have at most 1 non-zero bidprice_usd
        # take the largest
        unique_sessions = df.groupby('sessionid')['bidprice_usd'].max()
    else:
        # add up all bidprice_usd values for each sessionid
        unique_sessions = df.groupby('sessionid')['bidprice_usd'].sum()
    
    return sum(unique_sessions) / len(unique_sessions)

In [25]:
# VPT only applies to treatment group
# control group did not see an advertisement therefore cannot have bidprice_usd > 0
vpt_overall = calculate_vpt(data)
vpt_dup_overall = calculate_vpt(data, dedupe=False)

### Results

In [26]:
print(f' The overall VPT is ${vpt_overall:.3f}.\
    \n The overall VPT including likely duplicates is ${vpt_dup_overall:.3f}.\
      ')

 The overall VPT is $0.059.    
 The overall VPT including likely duplicates is $0.063.      


## Rev per User

In [53]:
rev_per_user = data[['group', 'cohort', 'userhash', 'value']].groupby(['group', 'cohort', 'userhash']).sum()
rev_grouped = rev_per_user.reset_index().groupby(['group', 'cohort'])['value'].agg(['count', 'sum']).reset_index()
rev_grouped['rev_per_user'] = rev_grouped['sum'] / rev_grouped['count']

#rev_grouped.to_csv('rev_grouped.csv')

rev_grouped

,group,cohort,count,sum,rev_per_user
0,control,202002,28609,60850.9711,2.126987
1,control,202003,23822,46576.1333,1.955173
2,control,202004,29184,49207.3774,1.686108
3,control,202005,18736,28557.0869,1.524183
4,control,202006,25889,28262.2000,1.091668
5,treatment,202002,68908,179530.3539,2.605363
6,treatment,202003,57137,152016.8204,2.660567
7,treatment,202004,115316,236159.9539,2.047937
8,treatment,202005,170452,773439.8038,4.537581
9,treatment,202006,233631,776511.0046,3.323664


As visualized in the slide deck, the treatment group provided even greater uplift on revenue per user than it did on conversion rate. Notably, the uplift **increased** as we moved further into the pandemic and spending (as noted in the diminishing conversion rates from these months) decreased.

I would focus on this uplift of revenue per user when pitching clients on greater ad spend.

# Metrics by Cohort

In [34]:
cohorts = data['cohort'].unique()
metrics = []

# iterate over cohorts and calculate metrics for each
for cohort in cohorts:
    
    # filter to cohort
    subset = data[data['cohort'] == cohort]
    
    # total unique users in cohort
    n = subset['userhash'].nunique()
    
    # total unique converted users in cohort
    n_conv = subset[subset['value'] > 0]['userhash'].nunique()
    
    # calculate conversion rate for each group
    # store unique users and unique converters per group
    cr_T, n_conv_T, n_T = calculate_conv_rate(data, 'treatment', {'cohort':cohort})
    cr_C, n_conv_C, n_C = calculate_conv_rate(data, 'control', {'cohort':cohort})
    
    # calculate VPT for group
    vpt = calculate_vpt(data, {'cohort':cohort})
    
    metrics.append([
        cohort, n_conv, n            # cohort level
        , n_conv_C, n_C, cr_C        # control group level
        , n_conv_T, n_T, cr_T        # treatment group level
        , vpt
    ])

In [35]:
# append overall metrics to list of cohort metrics
metrics.append([
    'overall', n_conv_overall, n_overall
    , n_conv_C_overall, n_C_overall, conv_rate_C_overall
    , n_conv_T_overall, n_T_overall, conv_rate_T_overall
    , vpt
])

cohort_metrics = pd.DataFrame(
    metrics
    , columns=['cohort', 'n_converters', 'n_users'
               , 'n_converters_C', 'n_users_C', 'conv_rate_C'
               , 'n_converters_T', 'n_users_T', 'conv_rate_T'
               , 'VPT']
)

# calculate uplift per cohort
cohort_metrics['uplift'] = (cohort_metrics['conv_rate_T'] - cohort_metrics['conv_rate_C'])/cohort_metrics['conv_rate_C']

# calculate absolute difference between control and treatment conversion rates
cohort_metrics['absolute_diff'] = cohort_metrics['conv_rate_T'] - cohort_metrics['conv_rate_C']

# export to .csv to create quick visualizations in Tableau
#cohort_metrics.to_csv('cohort_metrics.csv')

cohort_metrics = cohort_metrics.set_index('cohort')

In [36]:
cohort_metrics

,n_converters,n_users,n_converters_C,n_users_C,conv_rate_C,n_converters_T,n_users_T,conv_rate_T,VPT,uplift,absolute_diff
cohort,,,,,,,,,,,
202002,2741,97498,673,28609,0.023524,2068,68908,0.030011,0.071387,0.275759,0.006487
202003,2231,80943,551,23822,0.023130,1680,57137,0.029403,0.055409,0.271213,0.006273
202004,2975,144469,550,29184,0.018846,2425,115316,0.021029,0.051257,0.115846,0.002183
202005,3393,189176,293,18736,0.015638,3100,170452,0.018187,0.040757,0.162971,0.002549
202006,3875,259506,312,25889,0.012051,3563,233631,0.015251,0.073420,0.265453,0.003199
overall,15211,671795,2379,121750,0.019540,12833,576471,0.022261,0.073420,0.139266,0.002721


In [37]:
# select subset of cohort_metrics
metrics_simple = cohort_metrics[['conv_rate_C', 'conv_rate_T', 'uplift', 'absolute_diff', 'VPT']]

# format percentages
for m in ['conv_rate_C', 'conv_rate_T', 'uplift', 'absolute_diff']:
    metrics_simple.loc[:, m] = metrics_simple[m].map('{:.2%}'.format)

metrics_simple.loc[:, 'VPT'] = metrics_simple['VPT'].map('${:.2}'.format)

### Results

In [38]:
metrics_simple

,conv_rate_C,conv_rate_T,uplift,absolute_diff,VPT
cohort,,,,,
202002,2.35%,3.00%,27.58%,0.65%,$0.071
202003,2.31%,2.94%,27.12%,0.63%,$0.055
202004,1.88%,2.10%,11.58%,0.22%,$0.051
202005,1.56%,1.82%,16.30%,0.25%,$0.041
202006,1.21%,1.53%,26.55%,0.32%,$0.073
overall,1.95%,2.23%,13.93%,0.27%,$0.073


# Metrics by Vertical

intermediate values needed:

Rev Per User:

    sum(distinct value per sessionid)

VPT:

    count of unique sessionids
    sum of max(bidprice_usd) per sessionids

Conv Rates, Uplift:

    count(unique users) per group
    count(unique users who converted) per group

## Conv Rate and Uplift

In [40]:
converted = data[data['value'] > 0]
converted = converted.groupby(['cohort', 'verticalname', 'group'])['userhash'].count()
converted = converted.unstack(level = -1)
converted.columns = ['n_conv_C', 'n_conv_T']

all_users = data.groupby(['cohort', 'verticalname', 'group'])['userhash'].count()
all_users = all_users.unstack(level = -1)
all_users.columns = ['n_C', 'n_T']


uplift_by_vertical = all_users.merge(converted, how = 'left', on = ['cohort', 'verticalname'])

In [41]:
uplift_by_vertical['CR_C'] = uplift_by_vertical['n_conv_C'] / uplift_by_vertical['n_C']
uplift_by_vertical['CR_T'] = uplift_by_vertical['n_conv_T'] / uplift_by_vertical['n_T']

uplift_by_vertical = uplift_by_vertical.fillna(0)
uplift_by_vertical['uplift'] = np.where(uplift_by_vertical['CR_C'] > 0, (uplift_by_vertical['CR_T'] - uplift_by_vertical['CR_C']) / uplift_by_vertical['CR_C'], 0)

In [42]:
#uplift_by_vertical.to_csv('uplift_by_vertical.csv')

In [43]:
uplift_by_vertical
#uplift_by_vertical.pivot_table(index = ['cohort', 'verticalname'], columns = ['group', 'n', 'n_conv'])

n_C       n_T  n_conv_C  n_conv_T   
cohort verticalname                                                     
202002 Finance                     28.0      80.0       1.0       2.0  \
       Food and Beverage          750.0    2014.0      15.0      35.0   
       Loyalty and Samples          5.0      16.0       0.0       1.0   
       Media and Entertainment  11743.0   30401.0     237.0     662.0   
       Retail                    2854.0    8695.0     101.0     377.0   
       Sports and Fitness           0.0       2.0       0.0       1.0   
       Ticketing                11573.0   35596.0     270.0     856.0   
       Travel                    1729.0    4753.0      49.0     134.0   
202003 Finance                      0.0       4.0       0.0       0.0   
       Food and Beverage         1684.0    4808.0      39.0     110.0   
       Loyalty and Samples          5.0      18.0       0.0       0.0   
       Media and Entertainment  14996.0   38254.0     288.0     797.0   
       Retail                    3895.0   11844.0     140.0     504.0   
       Ticketing                 2289.0    7368.0      56.0     196.0   
       Travel                    1024.0    2806.0      28.0      73.0   
202004 Finance                      9.0      53.0       0.0       0.0   
       Food and Beverage         7218.0   35039.0     113.0     510.0   
       Loyalty and Samples          5.0      28.0       0.0       0.0   
       Media and Entertainment  17777.0   70089.0     292.0    1164.0   
       Retail                    3835.0   17881.0     140.0     702.0   
       Ticketing                  424.0    2501.0       5.0      47.0   
       Travel                      22.0     127.0       0.0       2.0   
202005 Finance                      5.0      34.0       0.0       0.0   
       Food and Beverage         7012.0   66132.0      79.0     727.0   
       Loyalty and Samples          2.0      32.0       0.0       0.0   
       Media and Entertainment   9102.0   88567.0     149.0    1470.0   
       Retail                    2193.0   25115.0      61.0     824.0   
       Ticketing                  448.0    5193.0       5.0      76.0   
       Travel                      19.0     198.0       0.0       4.0   
202006 Finance                      9.0     119.0       0.0       0.0   
       Food and Beverage         4056.0   37616.0      18.0     175.0   
       Loyalty and Samples          3.0      32.0       0.0       1.0   
       Media and Entertainment  14103.0  132532.0     125.0    1230.0   
       Retail                    6440.0   76893.0     153.0    1971.0   
       Ticketing                 1110.0   12895.0      14.0     165.0   
       Travel                     239.0    2414.0       2.0      21.0   

                                    CR_C      CR_T    uplift  
cohort verticalname                                           
202002 Finance                  0.035714  0.025000 -0.300000  
       Food and Beverage        0.020000  0.017378 -0.131082  
       Loyalty and Samples      0.000000  0.062500  0.000000  
       Media and Entertainment  0.020182  0.021776  0.078949  
       Retail                   0.035389  0.043358  0.225193  
       Sports and Fitness       0.000000  0.500000  0.000000  
       Ticketing                0.023330  0.024048  0.030753  
       Travel                   0.028340  0.028193 -0.005200  
202003 Finance                  0.000000  0.000000  0.000000  
       Food and Beverage        0.023159  0.022879 -0.012117  
       Loyalty and Samples      0.000000  0.000000  0.000000  
       Media and Entertainment  0.019205  0.020834  0.084837  
       Retail                   0.035944  0.042553  0.183891  
       Ticketing                0.024465  0.026602  0.087337  
       Travel                   0.027344  0.026016 -0.048569  
202004 Finance                  0.000000  0.000000  0.000000  
       Food and Beverage        0.015655  0.014555 -0.070270  
       Loyalty and Samples      0.000000  0.000000  0.000000  
 

## Value per Transaction

In [45]:
vpt_by_vertical = data.groupby(['cohort', 'verticalname', 'group', 'sessionid'])['bidprice_usd'].max()

vpt_by_vertical = vpt_by_vertical.reset_index()

In [46]:
#vpt_by_vertical[(vpt_by_vertical['cohort'] == 202002) & (vpt_by_vertical['verticalname'] == 'Loyalty and Samples')]

In [48]:
final_vpt_by_vertical = vpt_by_vertical.groupby(['cohort', 'verticalname', 'group']).agg({'sessionid': pd.Series.count, 'bidprice_usd': np.sum})
final_vpt_by_vertical.columns = ['sessions', 'bids_total']
final_vpt_by_vertical['VPT'] = final_vpt_by_vertical['bids_total'] / final_vpt_by_vertical['sessions']

final_vpt_by_vertical.reset_index().to_csv('vpt_by_vertical.csv')

In [49]:
final_vpt_by_vertical

sessions   bids_total       VPT
cohort verticalname        group                                     
202002 Finance             control          28      0.00000  0.000000
                           treatment        70      3.16540  0.045220
       Food and Beverage   control         750      0.00000  0.000000
                           treatment      1844     73.93339  0.040094
       Loyalty and Samples control           5      0.00000  0.000000
...                                        ...          ...       ...
202006 Retail              treatment     58599  15395.48628  0.262726
       Ticketing           control        1110      0.00000  0.000000
                           treatment     10121   1410.32850  0.139347
       Travel              control         239      0.00000  0.000000
                           treatment      2327     29.00841  0.012466

[70 rows x 3 columns]

## Revenue per User

In [ ]:
rpu_by_vertical = data.groupby(['cohort', 'verticalname', 'group'])[['userhash', 'value']].agg({'userhash': pd.Series.nunique, 'value': sum})
rpu_by_vertical.columns = ['users', 'revenue']

rpu_by_vertical['rpu'] = rpu_by_vertical['revenue'] / rpu_by_vertical['users']

In [ ]:
# rpu_by_vertical.reset_index().to_csv('rpu_by_vertical.csv')

# Does the campaign provide a positive or negative impact? (assuming Conversion Rate as primary metric)

Yes, the campaign's advertisements generated 14% uplift, lifting the conversion rate from 1.95% in the holdout (control) group to 2.23% in the treatment group.

At the cohort level, the treatment yielded uplift as well, ranging from 11.5% in April 2020 to 27.5% in February 2020.

The absolute change in conversion rates were small: at  most, the rate increased by 0.65% (from 2.35% to 3% in February).

# How confident are we in the positive or negative impact?

I assume a significance level of $\alpha$ = 0.05 for a 95% confidence interval.

Given the large samples of users, I expect the uplift to be statistically significant for all six cohorts as well as overall. Let's confirm it mathematically.

### Background on Hypothesis Test for Proportions

We meet the normal condition (we expect at least 10 conversions and at least 10 non-conversions in each sample).

I assume we meet the random and independent conditions (sample size is less than 10% of population of potential users).

$ H_0: CR_T = CR_C$

$H_A: CR_T \neq CR_C$

We always assume the treatment does not yield an effect (the null hypothesis). What is the probability that we would obtain the above results if the treatment did not yield an effect?

To quantify that probability, we calculate the test statistic Z:

$ z = \frac{CR_T - CR_C}{\sqrt{\frac{CR_T(1 - CR_T)}{n}}}$

where the denominator is the standard error of our sample proportion, $CR_T$.

Then we evaluate the likelihood of that z-statistic occurring (under null hypothesis assumption). We conduct a two-sided test since we want to know if our treatment conversion rate is higher **or** lower than the control.

$ p(z) = 2 * (1 - stats.norm.cdf(|z|)) $

In [16]:
def calc_p_value(cr_T, cr_C, n):
    
    std_error = sqrt(cr_T * (1 - cr_T) / n)
    z = (cr_T - cr_C) / std_error
    
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))
    
    return p_value

In [17]:
# calculate the p-value
cohort_metrics['p_value'] = cohort_metrics.apply(lambda row: calc_p_value(row['conv_rate_T'], row['conv_rate_C'], row['n_users']), axis=1)

In [18]:
# add to simplified metric view with formatting
metrics_simple.loc[:, 'p_value'] = cohort_metrics.loc[:, 'p_value'].map('{:.9%}'.format)

/var/folders/hz/5zk_qcfx4q19_bz870d0z8wr0000gn/T/ipykernel_91682/315451427.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metrics_simple.loc[:, 'p_value'] = cohort_metrics.loc[:, 'p_value'].map('{:.9%}'.format)


### p-values by cohort

In [19]:
metrics_simple

,conv_rate_C,conv_rate_T,uplift,absolute_diff,VPT,p_value
cohort,,,,,,
202002,2.35%,3.00%,27.58%,0.65%,$0.071,0.000000000%
202003,2.31%,2.94%,27.12%,0.63%,$0.055,0.000000000%
202004,1.88%,2.10%,11.58%,0.22%,$0.051,0.000000732%
202005,1.56%,1.82%,16.30%,0.25%,$0.041,0.000000000%
202006,1.21%,1.53%,26.55%,0.32%,$0.073,0.000000000%
overall,1.95%,2.23%,13.93%,0.27%,$0.073,0.000000000%


As expected, the large sample sizes lead to very, very small p-values. In other words, it is very unlikely we would obtain these results if the treatment conversion rate were the same as the control conversion rate. The results are statistically significant.

Thus, we can conclude with over 95% (in fact over 99%) confidence that the treatment conversion rate is higher than the control conversion rate. Whether the absolute difference is large enough to justify the cost of running the treatment will depend on the client.

# Data Exploration and Validation

In [1]:
file_path = 'holdout_data.csv'

test = pd.read_csv(file_path)
test.shape

NameError: name 'pd' is not defined

In [26]:
test.head()

,agegroup,bidprice_usd,campaigntimestamp,cohort,device,gender,group,sessionid,userhash,value,verticalname
0,26-30,0.0,2020-02-01 00:01:39,202002,mobile,f,treatment,37348334365,e7j+F6Pk3JiuQi+y3HI0xX7mzuSY5tTohpQM0FZIWPU=,NaN,Ticketing
1,36-40,0.0,2020-02-01 00:04:09,202002,Mobile,f,treatment,37348334755,217xmeBlQVPlC4vVXRdpSFG1uvbA2ET6BL/TaPAK7mo=,NaN,Media and Entertainment
2,31-35,0.0,2020-02-01 00:04:37,202002,Mobile,f,treatment,37349334794,YZm04BwARD+mj0R0MEYuKeAf4l1ZIRsge2LumlsnmTQ=,NaN,Media and Entertainment
3,26-30,0.0,2020-02-01 00:04:41,202002,Mobile,f,treatment,37348334834,TR25fI0N1lOqSu2hadffCZ+SNIA6iY9BHAnIXAVXuxM=,NaN,Ticketing
4,36-40,0.0,2020-02-01 00:04:54,202002,Other,f,treatment,37348334883,+ufGey+TPBU8o2SjY+AXrNvocVGXKjQ+H8pqq7jQ8y8=,NaN,Media and Entertainment


In [27]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 889091 entries, 0 to 889090
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   agegroup           889091 non-null  object 
 1   bidprice_usd       889091 non-null  float64
 2   campaigntimestamp  889091 non-null  object 
 3   cohort             889091 non-null  int64  
 4   device             888969 non-null  object 
 5   gender             798461 non-null  object 
 6   group              889091 non-null  object 
 7   sessionid          889091 non-null  object 
 8   userhash           889091 non-null  object 
 9   value              15935 non-null   float64
 10  verticalname       889091 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 74.6+ MB


All records contain most fields.

In [28]:
test.describe()

,bidprice_usd,cohort,value
count,889091.000000,889091.000000,15935.000000
mean,0.057920,202004.558184,150.293581
std,0.254488,2.443345,4787.571076
min,0.000000,202002.000000,0.565800
25%,0.000000,202004.000000,47.664300
50%,0.000000,202005.000000,71.062200
75%,0.000000,202006.000000,109.001600
max,5.507550,203902.000000,466466.000000


Check sets of possible values for each categorical variable to ensure they are well-formatted.

In [29]:
cats = ['agegroup', 'cohort', 'device', 'gender', 'group', 'verticalname']

for cat in cats:
    print (test[cat].value_counts().sort_values())

agegroup
65+        1598
56-65     25356
46-55     43915
41-45     83007
26-30    100159
36-40    126151
31-35    140313
18-25    368592
Name: count, dtype: int64
cohort
203902         1
202003     93444
202002    115940
202004    162761
202005    214188
202006    302757
Name: count, dtype: int64
device
Other       37672
Tablet      99562
mobile     110537
Desktop    171011
Mobile     470187
Name: count, dtype: int64
gender
m    154908
f    643553
Name: count, dtype: int64
group
control      132942
treatment    756149
Name: count, dtype: int64
verticalname
Sports and Fitness              2
Loyalty and Samples           158
Finance                       366
Travel                      14028
Ticketing                   83448
Retail                     167552
Food and Beverage          174612
Media and Entertainment    448925
Name: count, dtype: int64


Does the data contain duplicate records?

In [30]:
test_deduped = test.drop_duplicates()

In [31]:
print(f'Original -> Deduped Data: {test.shape[0]:,} -> {test_deduped.shape[0]:,}')
print(f'{(test.shape[0] - test_deduped.shape[0]):,} duplicate records removed')

Original -> Deduped Data: 889,091 -> 852,198
36,893 duplicate records removed


See DataQuality.ipynb for investigation into data duplication.

In [32]:
# how many conversions in the deduped data?
conv_pivot = pd.pivot_table(
    test_deduped
    , values = ['value']
    , index='group'
    , aggfunc = 'count'
    , margins = True
)

conv_pivot

,value
group,
control,2397
treatment,12904
All,15301


Can a single user belong to both control and treatment groups. -> Yes, see DataQuality.ipynb

Each sessionid belongs to only one group. But, one user can have multiple sessionids. So, if we group by sessionid to calculate conversion rate, we will overcount unique users.

Assumption: If a user has sessions under both treatment and control, the respective sessions contribute to both groups' metrics.

In [33]:
# check if value <= 0 for any record
# non-conversions should have NaN 'value': value <= 0 would be ambiguous
test[test['value'] <= 0]

,agegroup,bidprice_usd,campaigntimestamp,cohort,device,gender,group,sessionid,userhash,value,verticalname


The equation for Value per Transaction seems to imply that each unique sessionid will only have one bidprice_usd.

See DataQuality.ipynb for details on sessionids associated with multiple bidprice_usd values.

In [34]:
#unique_sessions = data[['sessionid', 'bidprice_usd', 'value']].groupby('sessionid').nunique()

sessions = test[['sessionid', 'bidprice_usd']].groupby('sessionid').sum()